# General Preamble Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display
import urllib.request
import tarfile
import io
import seaborn as sns
import warnings
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

# Additional code for GE Dataset

In [ ]:
# For GE, additional import code:
def load_tcga_data():
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00401/TCGA-PANCAN-HiSeq-801x20531.tar.gz'
    try:
        with urllib.request.urlopen(url) as response:
            with tarfile.open(fileobj=io.BytesIO(response.read()), mode="r:gz") as tar:
                data_path = 'TCGA-PANCAN-HiSeq-801x20531/data.csv'
                labels_path = 'TCGA-PANCAN-HiSeq-801x20531/labels.csv'
                data_file = tar.extractfile(data_path)
                X = pd.read_csv(data_file, index_col=0)
                labels_file = tar.extractfile(labels_path)
                labels_df = pd.read_csv(labels_file, index_col=0)
                y = labels_df['Class']
                return X, y
    except Exception as e:
        print(f"Failed to download or process data. Error: {e}")
        return None, None
X, y = load_tcga_data()

# Question 1
- Standardize the training and test sets using StandardScaler into (X_train_scaled). How many samples are there in your training dataset?
- Train a KNeighborsClassifier (n_neighbors=5) on the standardized, full-dimensional training data. Report the model's accuracy on the standardized test set. Also use the %%time magic command to measure and report the total time it takes to fit the model and make predictions.
- Now, do the above step and train 2 additional KNNs on only the first 300 samples, and the first 450 samples in the datasets. Plot the accuracy and time taken with 300, 450, and 600 on the x-axis and the corresponding values of time and accuracy on the y-axis.
- Does this plot allow you to guesstimate the time and accuracy when there are 50,000 patients in the dataset? What challenges does a distance-based algorithm like KNN still face on a scaled, but very high-dimensional, dataset?

In [ ]:
print("############ Assignment 7 Question 1 BEGIN ############")
# part a
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"Number of samples in training dataset: {X_train.shape[0]}")

# part b
knn_full = KNeighborsClassifier(n_neighbors=5)
knn_full.fit(X_train_scaled, y_train)
y_pred_full = knn_full.predict(X_test_scaled)
print(f"Accuracy on standardized test set: {accuracy_score(y_test, y_pred_full):.4f}")

# part c
sample_sizes = [300, 450, 600]
accuracies = []
times = []

for n in sample_sizes:
    X_train_n = X_train_scaled[:n]
    y_train_n = y_train.iloc[:n]
    start = time.time()
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_n, y_train_n)
    y_pred = knn.predict(X_test_scaled)
    elapsed = time.time() - start
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    times.append(elapsed)
    print(f"Accuracy {n} samples: {acc:.4f}, Time: {elapsed:.2f}s")

# Plot accuracy and time
fig, ax1 = plt.subplots()
color = 'tab:blue'
ax1.set_xlabel('Training Samples')
ax1.set_ylabel('Accuracy', color=color)
ax1.plot(sample_sizes, accuracies, marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Time (second)', color=color)
ax2.plot(sample_sizes, times, marker='o', color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('KNN: Training Size vs. Accuracy + Time')
plt.show()

# part d
print("While we could extrapolate the model training time is flattening out and will be around 0.5 seconds, the trend for model accuracy as train set size increases is less clear.")
print("This is because KNN faces challenges in high dimensions due to the curse of dimensionality, making distance less meaningful and computation expensive even with scaling. Especially with 50,000 patients, time and memory requirements would grow rapidly exponentially.")
print("############# Assignment 7 Question 1 END #############")


# Question 2
- Fit an instance of PCA on the standardized training data (X_train_scaled). Do not specify the number of components yet.
- Generate a "scree plot" by creating a bar chart of the explained_variance_ratio_ for the first 50 principal components. 
- Generate a cumulative explained variance plot for at least 500 components. On the same plot, add a horizontal line at y=0.90 to represent the 90% threshold.
- Based on your cumulative plot, approximately how many principal components are required to capture 90% of the total variance?

In [ ]:
print("############ Assignment 7 Question 2 BEGIN ############")

pca = PCA()
pca.fit(X_train_scaled)

explained_variance_ratio = pca.explained_variance_ratio_

n_components_to_plot = 50

plt.figure(figsize=(12, 6))
plt.bar(range(1, n_components_to_plot + 1), explained_variance_ratio[:n_components_to_plot], label="Individual Explained Variance Ratio")
plt.title(f"Screen Plot")
plt.xlabel("Principal Component Index")
plt.ylabel("Explained Variance Ratio")
plt.grid(axis="y", linestyle="--")
plt.xticks(range(0, n_components_to_plot + 1, 5))
plt.show()

n_components_to_plot = 500

cumulative_variance = np.cumsum(explained_variance_ratio)

plt.figure(figsize=(12, 6))
plt.plot(range(1, n_components_to_plot + 1), cumulative_variance[:n_components_to_plot], marker="o", linestyle="--",label="Cumulative Explained Variance")

threshold = 0.90
plt.axhline(y=threshold, linestyle="-", linewidth=2,label=f"{threshold*100:.0f}% Threshold")

n_components_90 = np.argmax(cumulative_variance >= threshold) + 1

plt.axvline(x=n_components_90, linestyle=":",label=f"Components for {threshold*100:.0f}%",)
    
plt.title(f"Cumulative Explained Variance for First {n_components_to_plot} PCs")
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.legend(loc="best")
plt.grid(True, linestyle="--")
plt.xticks(range(0, n_components_to_plot + 1, 5))
plt.ylim(0, 1.05)
plt.show()

print(f"Approximately {n_components_90} principal components are required to capture 90% of the total variance.")

print("############# Assignment 7 Question 2 END #############")

# Question 3
- Use the PCA model from Question 2 create the following 2D scatter plot. Color each point in the scatter plot according to its true cancer type (y_test). 
    - X-axis is PC1, y-axis PC2
    - X-axis is PC1, y-axis PC3 
    - X-axis PC2, y-axis PC3
    - X-axis PC1, y-axis PC4
- Based on your plots, how well does each combination of principal components separate the different cancer types? Why would it be impossible to create such an informative plot using any two of the original 20,000+ genes?

In [ ]:
print("############ Assignment 7 Question 3 BEGIN ############")

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
colors = encoder.fit_transform(y_test)

#Part A
X_pca = pca.transform(X_test_scaled)

plot_pairs = [
    (0, 1, 'PC1 vs PC2'),
    (0, 2, 'PC1 vs PC3'),
    (1, 2, 'PC2 vs PC3'),
    (0, 3, 'PC1 vs PC4')
]

for i, j, title in plot_pairs:
    plt.figure(figsize=(8, 6))
    
    scatter = plt.scatter(
        X_pca[:, i], # Select the X-axis PC
        X_pca[:, j], # Select the Y-axis PC
        c=colors,
        cmap='Accent'
    )

    legend_handles = []
    for idx, label in enumerate(encoder.classes_):
        legend_handles.append(plt.scatter([], [], marker='o', color=scatter.cmap(scatter.norm(idx)), label=label))
    plt.legend(handles=legend_handles, title="Cancer Type")

    plt.xlabel(f'Principal Component {i+1}')
    plt.ylabel(f'Principal Component {j+1}')
    plt.title(f'PCA Scatter Plot: {title}')
    plt.show()

#Part B
print("Each pair of principal components does somewhat okay at separating the different cancer types, usually able to isolate 2 types while the remaining 3 are overlapping.")
print("It's impossible to create as informative of a plot using pairs of the original genes because it's unlikely any random pair will truly capture the distinguishing features of each cancer type since they usually each rely on multiple different genes to be expressed.")

print("############# Assignment 7 Question 3 END #############")

# Question 4
- Create a new PCA instance, setting n_components to the number you identified in Question 2d to capture 90% of the variance. Fit this new PCA on the standardized training data and use it to transform both the training and test sets.
- Train the same KNeighborsClassifier from Question 1 on this new, lower-dimensional PCA-transformed training data again for 300, 450, and 600 samples.
- Report the accuracy and use %%time to report the total time for fitting and prediction and compare the accuracy and total time of this model to your baseline model in Question and draw the same plot relative to different sample sizes.
- What does this "apples-to-apples" comparison tell you about the specific advantages of PCA for an algorithm like KNN, especially if extrapolated to a situation where we might have 50,000 patients in the data?


In [ ]:
print("############ Assignment 7 Question 4 BEGIN ############")

#Part A
pca_q4 = PCA(n_components=n_components_90)
pca_q4.fit(X_train_scaled)
X_train_pca = pca_q4.transform(X_train_scaled)
X_test_pca = pca_q4.transform(X_test_scaled)

#Part B
sample_sizes = [300, 450, 600]
accuracies_q4 = []
times_q4 = []

print("KNN with PCA")
for n in sample_sizes:
    X_train_n = X_train_pca[:n]
    y_train_n = y_train.iloc[:n]
    start_q4 = time.time()
    knn_q4 = KNeighborsClassifier(n_neighbors=5)
    knn_q4.fit(X_train_n, y_train_n)
    y_pred_q4 = knn_q4.predict(X_test_pca)
    elapsed_q4 = time.time() - start
    acc_q4 = accuracy_score(y_test, y_pred_q4)
    accuracies_q4.append(acc_q4)
    times_q4.append(elapsed_q4)
#Part C (WIP; are these numbers correct? I'm not sure these are the results we should be getting)
    print(f"Accuracy {n} samples: {acc_q4:.4f}, Time: {elapsed_q4:.2f}s")

#compare with question 2's model:
print("\nKNN without PCA:")
for n in range(len(accuracies)):
    print(f"Accuracy {sample_sizes[n]} samples: {accuracies[n]:.4f}, Time: {times[n]:.4f}")

# Plot accuracy and time
fig, ax1 = plt.subplots()
color = 'tab:blue'
ax1.set_xlabel('Training Samples')
ax1.set_ylabel('Accuracy', color=color)
ax1.plot(sample_sizes, accuracies_q4, marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Time (second)', color=color)
ax2.plot(sample_sizes, times_q4, marker='o', color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('KNN w/ PCA: Training Size vs. Accuracy + Time')
plt.show()

#Part D (WIP)
print("This comparison tells us that a specific advantage of PCA for an algorithm like KNN is..."

print("############# Assignment 7 Question 4 END #############")

# Question 5
- Extract the loadings for the first principal component (PC1) from the components_ attribute of the PCA model fit in Question 2. 
- Find the top 10 genes with the largest absolute loading values and report their loading values. 
- Create a horizontal bar chart to visualize the loadings of these top 10 genes. 
- In a biological context, what does it mean for these specific genes to have high loading values on the first principal component? 

In [ ]:
print("############ Assignment 7 Question 5 BEGIN ############")

print("############# Assignment 7 Question 5 END #############")

# Question 6
- Use the PCA model from Question 4 (explaining 90% variance) to inverse_ transform your X_train_pca data back to the ~20k dimensions. Calculate the Mean Squared Error (MSE) between the original X_train_scaled and this new reconstructed data. What does this error represent?
- To visualize this, create a line plot showing the expression levels of the first 200 genes for the first patient in the original X_train_scaled data. On the same plot, show the expression levels for the same patient from the reconstructed data.

In [ ]:
print("############ Assignment 7 Question 6 BEGIN ############")

print("############# Assignment 7 Question 6 END #############")

# Question 7
- The Necessity of Scaling for PCA Explain, with reference to how PCA's algorithm is based on maximizing variance, why failing to scale features like gene expression levels would lead to a meaningless result. How does this differ from decision trees?


In [ ]:
print("############ Assignment 7 Question 7 BEGIN ############")
print("Since PCA's algorithm is based on maximizing variance, failing to scale features will mean that features with larger ranges of possible values will have a greater degree of variance, leading to PCA overprioritizing some features over others due to their scale rather than actual importance.")
print("This differs from decision trees which make splitting decisions per feature and thus don't require scaling.")
print("############# Assignment 7 Question 7 END #############")

# Question 8
- Interpretability: Like question 5b, now write out the top 10 genes of the first 5 principal components trained in Question 2 and their corresponding loading values. If you’re a data scientist with no biological background, and you are asked to explain the meaning of each of the principal components would you be able to explain it? How might a biologist use their expertise to explain each principal component better?

In [ ]:
print("############ Assignment 7 Question 8 BEGIN ############")
#write out the top 10 genes of the first 5 principal components from Q2 (WIP)
print("As someone without a biological background, the meaning of each principal component would be difficult to explain since its just capturing the vectors/PCs that capture the most variance.")
print("What each PC means because there is no latent label or meaning, it's a mathematical construct of orthogonal axes. However, a biologist might understand from the loadings of each PC what meaning is being derived from the original features.")
print("For example, a PC might have high loading values for a group of features that are related to a particular symptom and therefore the model uses that PC to classify cancer types that exhibit those symptoms."
print("############# Assignment 7 Question 8 END #############")

# Question 9
- Bioinformatics: This lab demonstrates a standard bioinformatics workflow: scaling -> PCA -> algorithmic processing. Explain why this approach is so powerful for finding meaningful patterns in massive genomic datasets and its importance for cancer research.

In [ ]:
print("############ Assignment 7 Question 9 BEGIN ############")
print("This bioinformatics approach towards fining meaningul patterns in massive datasets is powerful and important because it allows us to analyze high dimensionality datasets more easily and effectively. Trying to manually look at 10,000 genes for 10,000 patients to determine their cancer type is unfeasible and will be more error-prone than using algorithms like PCA to streamline the analysis. ")
print("############# Assignment 7 Question 9 END #############")